# Climate-Friendly Food Systems (CFFS) Labelling Project

### The University of British Columbia

#### Created by Silvia Huang

## Set up and Import Libraries

In [1]:
#install libraries if needed
#!pip3 install pdpipe

In [2]:
import numpy as np
import pandas as pd
import pdpipe as pdp
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest

In [3]:
#set the root path, change the directory into the project folder
os.chdir("/Users/silvia/cffs-label")

In [4]:
#enable reading data in the scrolling window 
pd.set_option("display.max_rows", None, "display.max_columns", None)

## Import Cleaned Datasets

In [5]:
Items = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Items_List.csv"))
Items.dtypes

ItemId             object
Description        object
CaseQty           float64
CaseUOM            object
PakQty            float64
PakUOM             object
InventoryGroup     object
dtype: object

In [6]:
Items.head()

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-4271,APPLES GRANNY SMITH,113.0,ea,1.0,CT,PRODUCE
1,I-4971,ARTICHOKE 1/4 SALAD CUT TFC,6.0,LG CAN,2.5,Kg,PRODUCE
2,I-2305,BACON PANCETTA,1.0,Kg,1.0,Kg,MEAT
3,I-1207,BAGUETTE FRENCH,24.0,each,1.0,CT,BREAD
4,I-17203,BALSAMIC GLAZE,2.0,bottle,2.0,L,FOOD - GROCERY


In [7]:
Ingredients = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Ingredients_List.csv"))
Ingredients.dtypes

IngredientId     object
Qty             float64
Uom              object
Conversion      float64
InvFactor       float64
Recipe           object
dtype: object

In [8]:
Ingredients.head()

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,P-18746,1.00,Kg,1.00000,1.0000,P-10241
1,I-3388,1.00,L,1.00000,0.3058,P-10496
2,I-4660,2.27,Kg,2.20462,0.6942,P-10496
3,I-3451,2.56,L,1.00000,1.2800,P-13933
4,I-4679,1.00,BUNCH,1.00000,0.0063,P-18318


In [9]:
Preps = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "Preps_List_Cleaned.csv"))
Preps.dtypes

PrepId             object
Description        object
PakQty            float64
PakUOM             object
InventoryGroup     object
StdQty            float64
StdUom             object
dtype: object

In [10]:
Preps.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-55516,BAKED|Lasagna|Spin Mushroom,5.550,Kg,NaN,5550.0,g
1,P-54666,BAKED|Pasta|Chicken Alfredo,6.176,Kg,NaN,6176.0,g
2,P-54664,BAKED|Pasta|Chorizo Penne,7.360,Kg,NaN,7360.0,g
3,P-56502,BAKED|Pasta|Shrimp Pesto,5.760,Kg,NaN,5760.0,g
4,P-56433,BATCH|Shrimp Remoulade,1.600,Kg,NaN,1600.0,g


In [11]:
Products = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Products_List.csv"))
Products.dtypes

ProdId         object
Description    object
SalesGroup     object
dtype: object

In [12]:
Products.head()

,ProdId,Description,SalesGroup
0,R-61778,ALF|Flatbread|4 Cheese,OK - AL FORNO
1,R-61780,ALF|Flatbread|Apple & Pancetta,OK - AL FORNO
2,R-61749,ALF|Flatbread|BBQ Chicken,OK - AL FORNO
3,R-50859,ALF|Flatbread|Bruschetta,OK - AL FORNO
4,R-50788,ALF|Flatbread|Caprese,OK - AL FORNO


In [13]:
Conversions = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Conversions_List.csv"))
Conversions.dtypes

ConversionId       object
Multiplier        float64
ConvertFromQty    float64
ConvertFromUom     object
ConvertToQty      float64
ConvertToUom       object
dtype: object

In [14]:
Conversions.head()

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,NaN,1.000000,1.0,XXX,1.00,L
1,NaN,0.877193,1.0,1.14L,1.14,L
2,NaN,0.666667,1.0,1.5L,1.50,L
3,NaN,0.571429,1.0,1.75 L,1.75,L
4,NaN,0.500000,1.0,2L,2.00,L


In [15]:
mapping = pd.read_csv(os.path.join(os.getcwd(), "data", "mapping", "Mapping.csv"))
mapping.dtypes

ItemId                                                     object
CategoryID                                                  int64
Description                                                object
CaseQty                                                   float64
CaseUOM                                                    object
PakQty                                                    float64
PakUOM                                                     object
InventoryGroup                                             object
Category ID                                                 int64
Food Category                                              object
Active Total Supply Chain Emissions (kg CO2 / kg food)    float64
dtype: object

In [16]:
mapping.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,I-57545,1,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,1,beef & buffalo meat,41.3463
1,I-10869,1,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,1,beef & buffalo meat,41.3463
2,I-7064,1,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,1,beef & buffalo meat,41.3463
3,I-37005,1,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,1,beef & buffalo meat,41.3463
4,I-37002,1,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,1,beef & buffalo meat,41.3463


### Unit Converter

In [17]:
#import standard unit conversion information
Std_Unit = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "standard_conversions.csv"))
Std_Unit.head()

,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,4.92890,1,tsp,4.92890,ml
1,14.78700,1,Tbsp,14.78700,ml
2,946.35000,1,qt,946.35000,ml
3,473.17625,1,pt,473.17625,ml
4,28.34950,1,oz,28.34950,g


In [18]:
liquid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'ml', 'ConvertFromUom'].tolist()
solid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'g', 'ConvertFromUom'].tolist()

In [19]:
#construct a standard unit converter
def std_converter(qty, uom):
    if uom in Std_Unit['ConvertFromUom'].tolist():
        multiplier = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'Multiplier']
        Qty = float(qty)*float(multiplier)
        Uom = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'ConvertToUom'].values[0]
    else:
        Qty = qty
        Uom = uom
    return (Qty, Uom)

In [20]:
#test the std_converter
assert std_converter(0.25,'lb') == (113.398, 'g')

In [21]:
#construct a unit converter for specific ingredients
spc_cov = list(filter(None, Conversions['ConversionId'].tolist()))

def spc_converter(ingre, qty, uom):
    if uom in liquid_unit + solid_unit:
        return std_converter(qty, uom)
    elif ingre in spc_cov:
        conversion = Conversions.loc[(Conversions['ConversionId'] == ingre) & (Conversions['ConvertFromUom'] == uom)
                                    & (Conversions['ConvertToUom'] == 'g')]
        multiplier = conversion['Multiplier']
        if multiplier.empty:
            return std_converter(qty, uom)
        else: 
            Qty = float(qty)/float(multiplier)
            Uom = conversion['ConvertToUom'].values[0]
            return (Qty, Uom)
    else:
        return std_converter(qty, uom)

In [22]:
#test the spc_converter
assert spc_converter('I-1120', 1, 'CT') == (50, 'g')

## GHG Factors Calculation for Preps

In [23]:
Preps['GHG Emission (g)'] = 0
Preps['GHG Emission(g)/StdUom'] = 0

In [24]:
def get_items_ghge_prep(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Preps.loc[index,'PrepId']]
    ghg = Preps.loc[index, 'GHG Emission (g)']
    weight = Preps.loc[index, 'StdQty']
    print('Index:', index, '\nIngres:\n', ingres)
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('I'):
            ghge = mapping.loc[mapping['ItemId'] == ingre, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
            #print(ghge)
            Qty = float(ingres.loc[idx,'Qty'])
            Uom = ingres.loc[idx,'Uom']
            if ingre in spc_cov:
                qty = spc_converter(ingre, Qty, Uom)[0]
                ghg += qty*float(ghge)
            else:
                qty = std_converter(Qty, Uom)[0]
                ghg += qty*float(ghge)
            print(ingre, Qty, Uom, qty, float(ghge), qty*float(ghge))
    Preps.loc[index, 'GHG Emission (g)'] = float(ghg)
    Preps.loc[index, 'GHG Emission(g)/StdUom'] = ghg/float(weight)

In [25]:
def get_preps_ghge_prep(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Preps.loc[index,'PrepId']]
    ghg = Preps.loc[index, 'GHG Emission (g)']
    weight = Preps.loc[index, 'StdQty']
    print('Index:', index, '\nIngres:\n', ingres)
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('P'):
            ghge = Preps.loc[Preps['PrepId'] == ingre, 'GHG Emission(g)/StdUom']
            #print(ghge)
            Qty = float(ingres.loc[idx,'Qty'])
            Uom = ingres.loc[idx,'Uom']
            if ingre in spc_cov:
                qty = spc_converter(ingre, Qty, Uom)[0]
                ghg += qty*float(ghge)
            else:
                qty = std_converter(Qty, Uom)[0]
                ghg += qty*float(ghge)
            print(ingre, Qty, Uom, qty, qty*float(ghge))
        #print('Final:\n', ghg, weight)
    Preps.loc[index, 'GHG Emission (g)'] = float(ghg)
    Preps.loc[index, 'GHG Emission(g)/StdUom'] = ghg/float(weight)

In [26]:
def link_preps(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Preps.loc[index,'PrepId']]
    ghg = Preps.loc[index, 'GHG Emission (g)']
    weight = Preps.loc[index, 'StdQty']
    if len(ingres) == 1:
        ingre = ingres.iloc[0]['IngredientId']
        if ingre.startswith('P'):
            print(ingres)
            ghge = Preps.loc[Preps['PrepId'] == ingre, 'GHG Emission(g)/StdUom']
            Qty = float(ingres.iloc[0]['Qty'])
            Uom = ingres.iloc[0]['Uom']
            if ingre in spc_cov:
                qty = spc_converter(ingre, Qty, Uom)[0]
                ghg = qty*float(ghge)
            else:
                qty = std_converter(Qty, Uom)[0]
                ghg = qty*float(ghge)
            print(ingre, ghge, Qty, Uom, qty, weight)
            print(ghg/float(weight))
    Preps.loc[index, 'GHG Emission (g)'] = float(ghg)
    Preps.loc[index, 'GHG Emission(g)/StdUom'] = ghg/float(weight)

In [27]:
for index, row in Preps.iterrows():
    get_items_ghge_prep(index , row)

Index: 0 
Ingres:
     IngredientId     Qty Uom  Conversion  InvFactor   Recipe
272      I-14179  500.00   g    0.001000    90.0901  P-55516
273       I-2141  100.00   g    0.001000    18.0180  P-55516
274       I-2146  500.00   g    0.001000    90.0901  P-55516
275       I-3408  600.00   g    0.002205   108.1081  P-55516
276      P-26076    2.25   L    1.000000     0.4054  P-55516
277      P-55515    1.60  Kg    1.000000     0.2883  P-55516
I-14179 500.0 g 500.0 8.9104 4455.2
I-2141 100.0 g 100.0 8.9104 891.04
I-2146 500.0 g 500.0 8.9104 4455.2
I-3408 600.0 g 600.0 1.5225 913.5
Index: 1 
Ingres:
     IngredientId    Qty Uom  Conversion  InvFactor   Recipe
242      I-14179  720.0   g       0.001   116.5803  P-54666
243       I-2141  160.0   g       0.001    25.9067  P-54666
244      I-41921  800.0   g       0.001   129.5337  P-54666
245      P-26216    2.4  Kg       1.000     0.3886  P-54666
246      P-34121  500.0   g       0.001    80.9585  P-54666
247       P-8196    1.6   L       1

Index: 23 
Ingres:
     IngredientId  Qty Uom  Conversion  InvFactor  Recipe
398      P-18746  1.0  Kg         1.0        1.0  P-9767
Index: 24 
Ingres:
     IngredientId  Qty Uom  Conversion  InvFactor   Recipe
147      P-50587  1.0  Kg         1.0        1.0  P-44429
Index: 25 
Ingres:
     IngredientId    Qty Uom  Conversion  InvFactor   Recipe
132      P-24452  11.55  Kg     2.20462      1.049  P-39167
Index: 26 
Ingres:
     IngredientId  Qty Uom  Conversion  InvFactor   Recipe
215       I-4971  2.6  Kg         1.0        1.3  P-50802
I-4971 2.6 Kg 2600.0 0.5029 1307.54
Index: 27 
Ingres:
     IngredientId  Qty Uom  Conversion  InvFactor   Recipe
131       I-3159  2.0   L         1.0     1.0811  P-38637
I-3159 2.0 L 2000.0 0.4306 861.1999999999999
Index: 28 
Ingres:
     IngredientId   Qty Uom  Conversion  InvFactor   Recipe
106       I-6829  2.84  Kg         1.0       1.42  P-32663
I-6829 2.84 Kg 2840.0 0.4306 1222.904
Index: 29 
Ingres:
     IngredientId  Qty Uom  Conversion  In

Index: 67 
Ingres:
     IngredientId    Qty Uom  Conversion  InvFactor   Recipe
367       I-3389  150.0  ml       0.001    50.0000  P-61756
368       I-3586   10.0   g       1.000     3.3333  P-61756
369      P-55083    1.0  Kg    1000.000     0.3333  P-61756
370      P-55084    1.0  Kg    1000.000     0.3333  P-61756
371       P-9851    1.0  Kg       1.000     0.3333  P-61756
I-3389 150.0 ml 150.0 5.6383 845.745
I-3586 10.0 g 10.0 9.3703 93.703
Index: 68 
Ingres:
     IngredientId    Qty Uom  Conversion  InvFactor   Recipe
98       P-34293    1.5  Kg     2.20462     0.3261  P-29170
99       P-44510  150.0   g     1.00000    32.6087  P-29170
100      P-44964    1.5  Kg  1000.00000     0.3261  P-29170
101      P-45403    1.5  Kg     1.00000     0.3261  P-29170
102      P-56459  750.0  ml     0.00100   163.0435  P-29170
Index: 69 
Ingres:
    IngredientId    Qty Uom  Conversion  InvFactor   Recipe
25       I-6443  400.0  ml    0.033814    13.3333  P-26062
26      P-18907  250.0   g    1.

Index: 102 
Ingres:
     IngredientId  Qty   Uom  Conversion  InvFactor   Recipe
228      I-15032  1.0  LOAF         1.0     0.0278  P-51992
I-15032 1.0 LOAF 1.0 1.5225 1.5225
Index: 103 
Ingres:
     IngredientId   Qty Uom  Conversion  InvFactor  Recipe
393       I-4589  22.7  Kg     2.20462     1.1762  P-9713
I-4589 22.7 Kg 22700.0 0.3062 6950.740000000001
Index: 104 
Ingres:
     IngredientId  Qty Uom  Conversion  InvFactor   Recipe
345       I-4605  1.0  Kg     2.20462     0.0017  P-57293
I-4605 1.0 Kg 1000.0 0.5029 502.90000000000003
Index: 105 
Ingres:
     IngredientId  Qty Uom  Conversion  InvFactor   Recipe
225      I-17159  1.0  Kg     2.20462        1.0  P-50966
I-17159 1.0 Kg 1000.0 0.6932 693.2
Index: 106 
Ingres:
     IngredientId   Qty Uom  Conversion  InvFactor   Recipe
148       I-4341  90.0   g    0.012598        1.8  P-44510
I-4341 90.0 g 90.0 0.5029 45.261
Index: 107 
Ingres:
    IngredientId  Qty Uom  Conversion  InvFactor   Recipe
97       I-4341  1.0  CT         

683      P-34880    5.0   g    0.001000        8.0  P-42320
I-11207 3.0 g 3.0 9.3703 28.1109
I-11240 400.0 g 400.0 8.9104 3564.16
I-1813 15.0 ml 15.0 0.3942 5.913
I-3640 1.0 g 1.0 9.3703 9.3703
I-3654 1.0 g 1.0 9.3703 9.3703
I-6026 4.0 g 4.0 0.44 1.76
I-6850 500.0 g 500.0 2.9782 1489.1000000000001
Index: 148 
Ingres:
     IngredientId  Qty Uom  Conversion  InvFactor   Recipe
725      P-39167  2.0  Kg         1.0       0.50  P-57227
726      P-41094  1.0  Kg         1.0       0.25  P-57227
727      P-41095  1.0  Kg         1.0       0.25  P-57227
Index: 149 
Ingres:
     IngredientId    Qty Uom  Conversion  InvFactor   Recipe
692       I-3141  300.0  ml         1.0    29.1262  P-53277
693       I-5983   10.0   L         1.0     0.9709  P-53277
694      P-57227    2.0  Kg         1.0     0.1942  P-53277
I-3141 300.0 ml 213.000000639 0.0 0.0
I-5983 10.0 L 10000.0 0.0 0.0
Index: 150 
Ingres:
     IngredientId  Qty Uom  Conversion  InvFactor   Recipe
640       I-4660  5.0  Kg     2.20462   

...

4003       I-6820  125.00   g       0.001    22.7273  P-37090
I-3336 1.0 L 1000.0 2.01 2009.9999999999998
I-3458 2.84 L 2840.0 1.5225 4323.9
I-3658 1.2 Kg 1200.0 1.6414 1969.68
I-3696 1.0 L 1000.0 1.93 1930.0
I-47774 90.0 ml 90.0 0.0 0.0
I-5983 250.0 ml 250.0 0.0 0.0
I-6026 1.0 g 1.0 0.44 0.44
I-6820 125.0 g 125.0 0.9734 121.67500000000001
Index: 654 
Ingres:
      IngredientId     Qty Uom  Conversion  InvFactor   Recipe
3979      I-11207   75.00   g    1.000000      7.500  P-36076
3980       I-1813  100.00  ml    0.033814     10.000  P-36076
3981       I-3388  250.00  ml    0.001000     25.000  P-36076
3982       I-3435  300.00   g    0.001013     30.000  P-36076
3983       I-3620   20.00   g    1.000000      2.000  P-36076
3984       I-3627   35.00   g    0.035274      3.500  P-36076
3985       I-3628   10.00   g    1.000000      1.000  P-36076
3986       I-3654   25.00   g    1.000000      2.500  P-36076
3987       I-3658  150.00   g    0.001000     15.000  P-36076
3988       I-3679

I-3284 5.0 ml 5.0 0.0 0.0
I-3625 2.0 g 2.0 9.3703 18.7406
I-3661 20.0 g 20.0 1.6414 32.828
Index: 673 
Ingres:
      IngredientId    Qty Uom  Conversion  InvFactor   Recipe
4411      I-17159   1.13  Kg     2.20462     0.8303  P-26378
4412       I-3388  45.00  ml     0.00100    33.0639  P-26378
4413       I-3692  30.00  ml     0.00100    22.0426  P-26378
4414      P-18907  10.00   g     1.00000     7.3475  P-26378
4415      P-26020  15.00   g     0.00100    11.0213  P-26378
4416      P-50500  15.00   g     1.00000    11.0213  P-26378
4417      P-56797  55.00   g     0.00100    40.4115  P-26378
I-17159 1.13 Kg 1130.0 0.6932 783.316
I-3388 45.0 ml 45.0 3.2401 145.8045
I-3692 30.0 ml 30.0 1.93 57.9
Index: 674 
Ingres:
      IngredientId   Qty    Uom  Conversion  InvFactor   Recipe
4516       I-4341   1.0     CT    1.000000     0.0056  P-53594
4517       I-4724  25.0      g    0.002205     0.1404  P-53594
4518       I-4772   1.0  BUNCH    1.000000     0.0056  P-53594
I-4341 1.0 CT 1.0 0.502

4452      P-26020   45.0   g    0.001000     0.0818  P-35338
I-3388 250.0 ml 250.0 3.2401 810.025
I-3695 20.0 ml 20.0 1.93 38.6
I-4463 80.0 g 80.0 0.622 49.76
I-4508 40.0 g 40.0 0.5029 20.116
I-4695 80.0 g 80.0 0.5029 40.232
Index: 693 
Ingres:
      IngredientId    Qty Uom  Conversion  InvFactor   Recipe
4461      I-16215    1.0  Kg       1.000      0.250  P-43918
4462       I-3392  500.0  ml       0.001    125.000  P-43918
4463       I-3590  100.0   g       0.001     25.000  P-43918
4464       I-3660  300.0   g       0.001     75.000  P-43918
4465       I-3696  750.0  ml       1.000    187.500  P-43918
4466      I-47774  500.0  ml       0.001    125.000  P-43918
4467       I-5983    1.5   L       1.000      0.375  P-43918
I-16215 1.0 Kg 1000.0 0.0 0.0
I-3392 500.0 ml 500.0 3.1509 1575.45
I-3590 100.0 g 100.0 4.2854 428.54
I-3660 300.0 g 300.0 1.6414 492.42
I-3696 750.0 ml 750.0 1.93 1447.5
I-47774 500.0 ml 500.0 0.0 0.0
I-5983 1.5 L 1500.0 0.0 0.0
Index: 694 
Ingres:
      Ingredient

I-47774 250.0 ml 250.0 0.0 0.0
I-6026 25.0 g 25.0 0.44 11.0
I-6028 225.0 g 225.0 1.6414 369.315
I-7631 20.0 g 20.0 9.3703 187.406
Index: 712 
Ingres:
      IngredientId     Qty Uom  Conversion  InvFactor   Recipe
4798      I-11211    10.0   g    1.000000     1.1111  P-44601
4799      I-11312    20.0   g    0.035274     2.2222  P-44601
4800       I-3654   150.0   g    1.000000    16.6667  P-44601
4801       I-4655   800.0   g    0.002205    88.8889  P-44601
4802       I-4657  1000.0   g    0.002205   111.1111  P-44601
4803       I-4708  1600.0   g    0.002205   177.7778  P-44601
4804       I-4796   350.0   g    0.002205    38.8889  P-44601
4805       I-4894  1400.0   g    0.002205   155.5556  P-44601
4806       I-4918     1.0  Kg    2.204620     0.1111  P-44601
4807      I-50750    10.0   g    0.035274     1.1111  P-44601
4808       I-6812    10.0   g    0.001000     1.1111  P-44601
I-11211 10.0 g 10.0 4.2854 42.854
I-11312 20.0 g 20.0 0.3942 7.884
I-3654 150.0 g 150.0 9.3703 1405.545
I

5106      P-57146  300.0   g     0.001000   107.1429  P-61678
I-2852 10.0 g 10.0 9.3703 93.703
I-2911 5.0 g 5.0 9.3703 46.8515
I-3390 60.0 ml 60.0 5.6383 338.298
I-3447 200.0 ml 200.0 0.5029 100.58
I-3630 10.0 g 10.0 4.2854 42.854
I-3695 20.0 ml 20.0 1.93 38.6
I-5983 500.0 ml 500.0 0.0 0.0
I-61420 2.0 Kg 2000.0 1.7542 3508.4
Index: 735 
Ingres:
      IngredientId   Qty Uom  Conversion  InvFactor   Recipe
5049       I-3979  40.0  Kg         1.0     1.3333  P-42293
5050      P-42292   6.0   L         1.0     0.2000  P-42293
I-3979 40.0 Kg 40000.0 4.3996 175984.00000000003
Index: 736 
Ingres:
      IngredientId    Qty Uom  Conversion  InvFactor   Recipe
5028       I-3658  908.0   g    0.001000      227.0  P-26331
5029      I-47774    2.0   L    1.000000        0.5  P-26331
5030       I-4796   20.0   g    0.002205        5.0  P-26331
5031       I-6443  720.0  ml    0.033814      180.0  P-26331
5032       I-7388    4.0  Kg    1.000000        1.0  P-26331
5033      P-26020   60.0   g    0.00

In [28]:
for index, row in Preps.iterrows():
    link_preps(index, row)

   IngredientId    Qty Uom  Conversion  InvFactor   Recipe
12      P-26514  500.0   g         1.0     1000.0  P-24750
P-26514 107    0.009144
Name: GHG Emission(g)/StdUom, dtype: float64 500.0 g 500.0 500.0
0.009143636363636364
    IngredientId   Qty Uom  Conversion  InvFactor   Recipe
351      P-56490  10.0  Kg         1.0     1.1765  P-58450
P-56490 37    4.230695
Name: GHG Emission(g)/StdUom, dtype: float64 10.0 Kg 10000.0 8500.0
4.977287889273358
    IngredientId   Qty Uom  Conversion  InvFactor   Recipe
146      P-19175  11.5  Kg         1.0     1.0445  P-43440
P-19175 123    0.427753
Name: GHG Emission(g)/StdUom, dtype: float64 11.5 Kg 11500.0 11010.0
0.4467902758855586
    IngredientId   Qty Uom  Conversion  InvFactor   Recipe
348      P-19175  11.5  Kg         1.0     1.0455  P-57499
P-19175 123    0.427753
Name: GHG Emission(g)/StdUom, dtype: float64 11.5 Kg 11500.0 11000.0
0.44719644886363635
    IngredientId  Qty Uom  Conversion  InvFactor  Recipe
397      P-18746  1.0  Kg  

     IngredientId   Qty Uom  Conversion  InvFactor   Recipe
3633      P-46076  20.0   g       0.001        1.0  P-49552
P-46076 183    28.09
Name: GHG Emission(g)/StdUom, dtype: float64 20.0 g 20.0 20.0
28.089999999999996
     IngredientId    Qty Uom  Conversion  InvFactor   Recipe
4100      P-50692  100.0   g         1.0     0.7692  P-50691
P-50692 229    1.85154
Name: GHG Emission(g)/StdUom, dtype: float64 100.0 g 100.0 130.0
1.4242615384615385


In [29]:
for index, row in Preps.iterrows():
    get_preps_ghge_prep(index, row)

Index: 0 
Ingres:
     IngredientId     Qty Uom  Conversion  InvFactor   Recipe
272      I-14179  500.00   g    0.001000    90.0901  P-55516
273       I-2141  100.00   g    0.001000    18.0180  P-55516
274       I-2146  500.00   g    0.001000    90.0901  P-55516
275       I-3408  600.00   g    0.002205   108.1081  P-55516
276      P-26076    2.25   L    1.000000     0.4054  P-55516
277      P-55515    1.60  Kg    1.000000     0.2883  P-55516
P-26076 2.25 L 2250.0 7696.292507462687
P-55515 1.6 Kg 1600.0 480.03478260869565
Index: 1 
Ingres:
     IngredientId    Qty Uom  Conversion  InvFactor   Recipe
242      I-14179  720.0   g       0.001   116.5803  P-54666
243       I-2141  160.0   g       0.001    25.9067  P-54666
244      I-41921  800.0   g       0.001   129.5337  P-54666
245      P-26216    2.4  Kg       1.000     0.3886  P-54666
246      P-34121  500.0   g       0.001    80.9585  P-54666
247       P-8196    1.6   L       1.000     0.2591  P-54666
P-26216 2.4 Kg 2400.0 2161.7875862

Index: 42 
Ingres:
     IngredientId    Qty Uom  Conversion  InvFactor   Recipe
305       I-1813   20.0   g    0.032787     0.0351  P-56492
306       I-3348   80.0   g    0.000947     0.1404  P-56492
307       I-3624    2.0   g    0.001000     0.0035  P-56492
308      P-18907    3.0   g    1.000000     0.0053  P-56492
309      P-41095   50.0   g    0.001000     0.0877  P-56492
310      P-43440   25.0   g    0.001000     0.0439  P-56492
311      P-50429    5.0   g    1.000000     0.0088  P-56492
312      P-53735   30.0   g    1.000000     0.0526  P-56492
313      P-56952  340.0   g    0.001000     0.5965  P-56492
P-18907 3.0 g 3.0 6.240777551020409
P-41095 50.0 g 50.0 42.49963256907702
P-43440 25.0 g 25.0 22.33951379427793
P-50429 5.0 g 5.0 3.9114444444444447
P-53735 30.0 g 30.0 16.763333333333335
P-56952 340.0 g 340.0 1693.132
Index: 43 
Ingres:
     IngredientId    Qty Uom  Conversion  InvFactor  Recipe
381       I-1813  500.0  ml    0.033814      400.0  P-7139
382       I-3623    6.0

289       I-3368  1.0   L      1000.0        0.2  P-56455
Index: 75 
Ingres:
     IngredientId   Qty Uom  Conversion  InvFactor   Recipe
111       I-2141   2.0  lb    0.453592       0.25  P-33520
112      P-18907  20.0   g    1.000000       2.50  P-33520
113      P-26020  60.0   g    0.001000       7.50  P-33520
114      P-33519   8.0   L    1.000000       1.00  P-33520
P-18907 20.0 g 20.0 41.60518367346939
P-26020 60.0 g 60.0 74.74751999999998
P-33519 8.0 L 8000.0 18089.615999999998
Index: 76 
Ingres:
     IngredientId    Qty Uom  Conversion  InvFactor   Recipe
324       I-2141    1.3  Kg       1.000      0.065  P-56503
325       I-2217   13.0   L       1.000      0.650  P-56503
326       I-2697    4.0  lb     453.592      0.200  P-56503
327       I-6820  100.0   g       0.001      5.000  P-56503
328      P-18907   30.0   g       1.000      1.500  P-56503
329      P-32661    6.0   L       1.000      0.300  P-56503
330      P-48489  200.0   g       0.001     10.000  P-56503
P-18907 30.

Index: 124 
Ingres:
   IngredientId   Qty Uom  Conversion  InvFactor   Recipe
5       I-4793  10.0  Kg     2.20462     1.2048  P-18746
Index: 125 
Ingres:
   IngredientId   Qty Uom  Conversion  InvFactor   Recipe
3       I-3451  2.56   L         1.0       1.28  P-13933
Index: 126 
Ingres:
     IngredientId  Qty Uom  Conversion  InvFactor   Recipe
150       I-4983  1.0  Kg    35.27399     1.5385  P-44901
Index: 127 
Ingres:
     IngredientId  Qty Uom  Conversion  InvFactor   Recipe
346      I-17593  1.0  lb         1.0     0.0026  P-57294
Index: 128 
Ingres:
     IngredientId    Qty Uom  Conversion  InvFactor   Recipe
208       I-4919  454.0   g    0.002205     1.5133  P-50676
Index: 129 
Ingres:
     IngredientId   Qty Uom  Conversion  InvFactor   Recipe
207       I-4933  10.0  Kg     2.20462     1.1111  P-50587
Index: 130 
Ingres:
    IngredientId  Qty Uom  Conversion  InvFactor   Recipe
11       I-4381  1.0  lb         1.0     1.1111  P-24452
Index: 131 
Ingres:
     IngredientId  Qt

830       I-5983    2.0   L       1.000     0.0833  P-26631
Index: 165 
Ingres:
     IngredientId    Qty Uom  Conversion  InvFactor   Recipe
868       I-1813  125.0  ml    0.033814       50.0  P-50739
869      I-29389    2.5  Kg    1.000000        1.0  P-50739
870       I-2979    5.0   g    1.000000        2.0  P-50739
871       I-3284   30.0  ml    0.002000       12.0  P-50739
872       I-3660  325.0   g    0.001000      130.0  P-50739
873       I-5983  250.0  ml    0.001000      100.0  P-50739
874      P-50495  125.0  ml    0.001000       50.0  P-50739
P-50495 125.0 ml 125.0 60.837500000000006
Index: 166 
Ingres:
     IngredientId    Qty Uom  Conversion  InvFactor   Recipe
818       I-1813  125.0  ml    0.033814    48.0769  P-26063
819       I-3284   30.0  ml    0.002000    11.5385  P-26063
820       I-3660  325.0   g    0.001000   125.0000  P-26063
821       I-4531    1.8  Kg    2.204620     0.6923  P-26063
822       I-5983  250.0  ml    0.001000    96.1538  P-26063
823      P-50495

...

3608      P-28759  43.0   g       0.001    28.6667  P-43858
P-28759 43.0 g 43.0 221.72334526666668
Index: 559 
Ingres:
      IngredientId   Qty Uom  Conversion  InvFactor   Recipe
3525      P-33869  30.0   g    0.001000       30.0  P-28373
3526      P-35258  10.0   g    0.001000       10.0  P-28373
3527      P-42425  25.0   g    1.000000       25.0  P-28373
3528      P-42730  20.0   g    1.000000       20.0  P-28373
3529      P-44475  10.0   g    1.000000       10.0  P-28373
3530      P-44652  25.0   g    0.001000       25.0  P-28373
3531      P-46862  40.0   g    0.001000       40.0  P-28373
3532      P-50840  25.0   g    0.002205       25.0  P-28373
P-33869 30.0 g 30.0 36.797560975609755
P-35258 10.0 g 10.0 8.974656846049045
P-42425 25.0 g 25.0 13.107500000000002
P-42730 20.0 g 20.0 17.415999999999997
P-44475 10.0 g 10.0 6.297999999999999
P-44652 25.0 g 25.0 20.95416666666667
P-46862 40.0 g 40.0 30.808888888888887
P-50840 25.0 g 25.0 15.712948142154506
Index: 560 
Ingres:
      Ingre

3662       I-4787  1.0  Kg     2.20462     0.0014  P-51535
Index: 596 
Ingres:
      IngredientId   Qty Uom  Conversion  InvFactor   Recipe
3633      P-46076  20.0   g       0.001        1.0  P-49552
P-46076 20.0 g 20.0 561.8
Index: 597 
Ingres:
      IngredientId  Qty Uom  Conversion  InvFactor   Recipe
3564       I-3448  4.0   L         1.0      2.439  P-33869
Index: 598 
Ingres:
      IngredientId  Qty     Uom  Conversion  InvFactor   Recipe
3473       I-3159  1.0  bottle         1.0      0.001  P-22247
Index: 599 
Ingres:
      IngredientId    Qty Uom  Conversion  InvFactor   Recipe
3627       I-3965    4.0  Kg         1.0     0.0571  P-45255
3628      I-51440  560.0   g         1.0     8.0000  P-45255
Index: 600 
Ingres:
      IngredientId   Qty Uom  Conversion  InvFactor   Recipe
3611       I-8363  20.0  Kg         1.0     1.1111  P-44675
Index: 601 
Ingres:
      IngredientId  Qty Uom  Conversion  InvFactor   Recipe
3618       I-4637  1.0  Kg     2.20462       1.25  P-44916
Inde

Index: 632 
Ingres:
      IngredientId   Qty Uom  Conversion  InvFactor   Recipe
4018       I-4558   9.0  Kg     2.20462      1.125  P-43341
4019      P-41468  16.0   L     1.00000      2.000  P-43341
P-41468 16.0 L 16000.0 14790.255466666666
Index: 633 
Ingres:
      IngredientId   Qty Uom  Conversion  InvFactor   Recipe
4190       I-5983  10.0   L         1.0        5.0  P-56890
4191       I-6026  30.0   g         1.0       15.0  P-56890
4192       I-8864   1.0  Kg      1000.0        0.5  P-56890
Index: 634 
Ingres:
      IngredientId  Qty Uom  Conversion  InvFactor   Recipe
4011      I-11185  1.0  Kg    35.27399        0.5  P-41619
4012       I-3642  5.0   g     1.00000        2.5  P-41619
4013       I-6026  5.0   g     1.00000        2.5  P-41619
Index: 635 
Ingres:
      IngredientId   Qty    Uom  Conversion  InvFactor  Recipe
4247       I-4679  20.0  BUNCH         1.0     5.0684  P-9733
Index: 636 
Ingres:
      IngredientId    Qty Uom  Conversion  InvFactor   Recipe
4125       I

Index: 676 
Ingres:
      IngredientId    Qty Uom  Conversion  InvFactor  Recipe
4594       I-3388  125.0  ml       0.001     4.6296  P-9742
4595      P-18907   40.0   g       1.000     1.4815  P-9742
4596       P-9863   30.0  Kg       1.000     1.1111  P-9742
P-18907 40.0 g 40.0 83.21036734693878
P-9863 30.0 Kg 30000.0 12562.5
Index: 677 
Ingres:
      IngredientId    Qty Uom  Conversion  InvFactor   Recipe
4476      I-37478    4.0   g    1.000000     3.3333  P-48487
4477       I-4657   10.0   g    0.002205     8.3333  P-48487
4478      P-18907    6.0   g    1.000000     5.0000  P-48487
4479      P-44429  900.0   g    0.001000   750.0000  P-48487
P-18907 6.0 g 6.0 12.481555102040819
P-44429 900.0 g 900.0 1386.4
Index: 678 
Ingres:
      IngredientId   Qty Uom  Conversion  InvFactor   Recipe
4474       I-3223  3.00   L     1.00000     0.6667  P-46842
4475      I-36672  2.25  Kg     2.20462     0.5000  P-46842
Index: 679 
Ingres:
      IngredientId    Qty Uom  Conversion  InvFactor   Re

P-53277 5.0 L 5000.0 623.3542830411503
Index: 719 
Ingres:
      IngredientId    Qty Uom  Conversion  InvFactor   Recipe
4773      I-16215  169.0   g       0.001     42.250  P-30552
4774      I-30504   22.0   g       1.000      5.500  P-30552
4775       I-3388   30.0  ml       0.001      7.500  P-30552
4776       I-3392   30.0  ml       0.001      7.500  P-30552
4777       I-3632    5.0   g       1.000      1.250  P-30552
4778       I-3639    5.0   g       1.000      1.250  P-30552
4779       I-3643    4.5   g       0.001      1.125  P-30552
4780       I-3660  600.0   g       0.001    150.000  P-30552
4781      I-47774  500.0  ml       0.001    125.000  P-30552
4782      P-26020   90.0   g       0.001     22.500  P-30552
4783      P-32673    1.6   L       1.000      0.400  P-30552
P-26020 90.0 g 90.0 112.12127999999998
P-32673 1.6 L 1600.0 0.0
Index: 720 
Ingres:
      IngredientId    Qty Uom  Conversion  InvFactor   Recipe
4851      I-10882  160.0  ml       0.001      128.0  P-55484
4

In [30]:
Preps

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom,GHG Emission (g),GHG Emission(g)/StdUom
0,P-55516,BAKED|Lasagna|Spin Mushroom,5.550,Kg,NaN,5550.000000,g,18891.267290,3.403832
1,P-54666,BAKED|Pasta|Chicken Alfredo,6.176,Kg,NaN,6176.000000,g,17657.751270,2.859092
2,P-54664,BAKED|Pasta|Chorizo Penne,7.360,Kg,NaN,7360.000000,g,22177.046905,3.013186
3,P-56502,BAKED|Pasta|Shrimp Pesto,5.760,Kg,NaN,5760.000000,g,29040.084386,5.041681
4,P-56433,BATCH|Shrimp Remoulade,1.600,Kg,NaN,1600.000000,g,12750.771772,7.969232
5,P-50500,CHIFFONADE|Basil,190.000,g,NaN,190.000000,g,100.580000,0.529368
6,P-24750,CHOPPED|Cilantro,0.500,Kg,NaN,500.000000,g,9.143636,0.018287
7,P-53735,CHOPPED|pickle,900.000,g,NaN,900.000000,g,502.900000,0.558778
8,P-58450,COOKED|Bahn Mi Chicken,8.500,Kg,NaN,8500.000000,g,84613.894118,9.954576
9,P-26068,COOKED|Caramelized Onion,1.200,Kg,PREP,1200.000000,g,3010.091750,2.508410


## GHGe Calculation for Products

In [31]:
Products['GHG Emission (g)'] = np.nan
Products['Weight (g)'] = np.nan

In [32]:
for index, row in Products.iterrows():
    ghg = 0
    weight = 0
    Products.loc[index, 'GHG Emission (g)'] = ghg
    Products.loc[index, 'Weight (g)'] = ghg

In [33]:
ingres = Ingredients.loc[Ingredients['Recipe'] == Products.loc[107,'ProdId']]
ingres

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
1487,P-47775,150.0,g,0.001,1.0,R-25415
1488,P-50380,35.0,ml,0.001,1.0,R-25415
1489,P-50651,90.0,g,0.001,1.0,R-25415
1490,P-51582,20.0,g,0.001,1.0,R-25415
1491,P-61660,20.0,g,1.000,1.0,R-25415
1492,P-7736,120.0,g,0.001,1.0,R-25415


In [34]:
def get_items_ghge(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Products.loc[index,'ProdId']]
    #print(ingres)
    ghg = Products.loc[index, 'GHG Emission (g)']
    weight = Products.loc[index, 'Weight (g)']
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('I'):
            ghge = mapping.loc[mapping['ItemId'] == ingre, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
            Qty = float(ingres.loc[idx,'Qty'])
            Uom = ingres.loc[idx,'Uom']
            if ingre in Conversions['ConversionId'].tolist():
                qty = spc_converter(ingre, Qty, Uom)[0]
                weight += qty
                ghg += qty*float(ghge)
            else:
                qty = std_converter(Qty, Uom)[0]
                weight += qty
                ghg += qty*float(ghge)
            #print(ingre, Qty, Uom, qty)
    Products.loc[index, 'GHG Emission (g)'] = float(ghg)
    Products.loc[index, 'Weight (g)'] = float(weight)

In [35]:
def get_preps_ghge(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Products.loc[index,'ProdId']]
    ghg = Products.loc[index, 'GHG Emission (g)']
    weight = Products.loc[index, 'Weight (g)']
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('P'):
            ghge = Preps.loc[Preps['PrepId'] == ingre, 'GHG Emission(g)/StdUom']
            Qty = float(ingres.loc[idx,'Qty'])
            Uom = ingres.loc[idx,'Uom']
            if ingre in Conversions['ConversionId'].tolist():
                qty = spc_converter(ingre, Qty, Uom)[0]
                weight += qty
                ghg += qty*float(ghge)
            else:
                qty = std_converter(Qty, Uom)[0]
                weight += qty
                ghg += qty*float(ghge)
    Products.loc[index, 'GHG Emission (g)'] = float(ghg)
    Products.loc[index, 'Weight (g)'] = float(weight)

In [36]:
def get_products_ghge(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Products.loc[index,'ProdId']]
    ghg = Products.loc[index, 'GHG Emission (g)']
    weight = Products.loc[index, 'Weight (g)']
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('R'):
            ghge = Products.loc[Products['ProdId'] == ingre, 'GHG Emission (g)']
            Weight = Products.loc[Products['ProdId'] == ingre, 'Weight (g)']
            Qty = float(ingres.loc[idx,'Qty'])
            ghg += Qty*float(ghge)
            weight += Qty*float(Weight)
    Products.loc[index, 'GHG Emission (g)'] = float(ghg)
    Products.loc[index, 'Weight (g)'] = float(weight)

In [37]:
for index, row in Products.iterrows():
    get_items_ghge(index , row)

In [38]:
for index, row in Products.iterrows():
    get_preps_ghge(index, row)

In [39]:
for index, row in Products.iterrows():
    get_products_ghge(index, row)

In [40]:
Products['GHG Emission (Kg)'] = round(0.001*Products['GHG Emission (g)'], 4)
Products['GHG Emission (Kg) / 100g'] = round(0.1*Products['GHG Emission (g)']/Products['Weight (g)'], 6)

In [41]:
Products

,ProdId,Description,SalesGroup,GHG Emission (g),Weight (g),GHG Emission (Kg),GHG Emission (Kg) / 100g
0,R-61778,ALF|Flatbread|4 Cheese,OK - AL FORNO,1229.947500,185.000000,1.2299,0.664836
1,R-61780,ALF|Flatbread|Apple & Pancetta,OK - AL FORNO,756.345750,140.000000,0.7563,0.540247
2,R-61749,ALF|Flatbread|BBQ Chicken,OK - AL FORNO,1023.874995,245.000000,1.0239,0.417908
3,R-50859,ALF|Flatbread|Bruschetta,OK - AL FORNO,522.037267,215.000000,0.5220,0.242808
4,R-50788,ALF|Flatbread|Caprese,OK - AL FORNO,629.237000,233.000000,0.6292,0.270059
5,R-61781,ALF|Flatbread|Chick Artichoke,OK - AL FORNO,999.038200,245.000000,0.9990,0.407771
6,R-50858,ALF|Flatbread|Greek,OK - AL FORNO,426.879716,200.000000,0.4269,0.213440
7,R-61498,ALF|Flatbread|Hawaii Five-0,OK - AL FORNO,970.671530,200.000000,0.9707,0.485336
8,R-50562,ALF|Flatbread|Italian,OK - AL FORNO,593.347237,141.000000,0.5933,0.420814
9,R-56337,ALF|Flatbread|Mediterranean,OK - AL FORNO,313.698698,145.000000,0.3137,0.216344


In [42]:
path = os.path.join(os.getcwd(), "data", "final", "Recipes GHG.csv")
Products.to_csv(path, index = False, header = True)